### Importing Libraries

In [1]:
# Uncomment the below line to install libraries
# !pip -q install pandas requests bs4 threading multiprocessing

In [2]:
import os
import pandas as pd
import threading, multiprocessing
import requests
from bs4 import BeautifulSoup

### Grand Prix Data Frame

In [3]:
# Creating a data frame for the rounds, Grands Prix, and number of laps
grand_prix_df = pd.DataFrame({
    "round_num": [1, 2, 3, 4, 5],
    "grand_prix": ["Bahrain", "Saudi Arabia", "Australia", "Japan", "China"],
    "laps": [57, 50, 58, 53, 56]
})

# Setting the round number as the data frame index
grand_prix_df.set_index("round_num", inplace=True)
grand_prix_df.head()

,grand_prix,laps
round_num,,
1,Bahrain,57
2,Saudi Arabia,50
3,Australia,58
4,Japan,53
5,China,56


In [4]:
file_check = False
file_rounds = set()

if os.path.exists("lap_times.csv"):
    file_rounds = set(pd.read_csv("lap_times.csv")["round"])
    file_check = True

# Returning the rounds in round_list that are not on file
set_diff = list(set(grand_prix_df.index.values).difference(file_rounds))
file_check, set_diff[:5]

(False, [1, 2, 3, 4, 5])

### Constructor Data Frame

In [5]:
# Creating a dictionary with each constructor's drivers
constructor_driver_dict = {
    "Alpine": ["GAS", "OCO"], "Aston Martin": ["ALO", "STR"], "Ferrari": ["LEC", "SAI"],
    "Haas": ["HUL", "MAG"], "Kick Sauber": ["BOT", "ZHO"], "McLaren": ["NOR", "PIA"],
    "Mercedes": ["HAM", "RUS"], "Racing Bulls": ["RIC", "TSU"], "Red Bull": ["PER", "VER"],
    "Williams": ["ALB", "SAR"]
}

pair = []

for constructor, drivers in constructor_driver_dict.items():
    pair.append([constructor, drivers[0]])
    pair.append([constructor, drivers[1]])

# Converting the dictionary to a data frame
constructor_df = pd.DataFrame(pair, columns=["constructor", "driver"])
constructor_df.head()

,constructor,driver
0,Alpine,GAS
1,Alpine,OCO
2,Aston Martin,ALO
3,Aston Martin,STR
4,Ferrari,LEC


### Lap Time Data Frame

In [6]:
lap_time_df = pd.DataFrame(columns=["round", "grand_prix", "driver",
                                    "lap", "position", "time"])

In [7]:
def grand_prix_loop(round_num, grand_prix, lap) -> None:
    url = f"https://ergast.com/api/f1/2023/{round_num}/laps/{lap}.xml"
    response = requests.get(url).content
    soup = BeautifulSoup(response, features="xml")
    timing_list = soup.findAll("Timing")